In [ ]:
!pip install nltk matplotlib pandas textstat fonemas

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.corpus import stopwords

# Descargar solo una vez
nltk.download('stopwords')
stopwords_esp = set(stopwords.words('spanish'))

import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from fonemas import Transcription
import textstat
import re
import os
import pandas as pd
from IPython.display import display


# Preprocesamiento inicial

In [ ]:
import zipfile
import os

def descomprimir_zip(ruta_zip, carpeta_destino):
    # Verificar si el archivo existe
    if not os.path.exists(ruta_zip):
        print(f"El archivo {ruta_zip} no existe.")
        return

    # Crear carpeta destino si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Abrir y extraer el zip
    with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
        zip_ref.extractall(carpeta_destino)
        print(f"Archivo descomprimido en: {carpeta_destino}")

# Ejemplo de uso
ruta_zip = "cantemist.zip"            # Ruta al archivo zip
carpeta_destino = "cantemist"  # Carpeta donde se extraerá
descomprimir_zip(ruta_zip, carpeta_destino)


In [ ]:

def cargar_textos_carpeta(carpeta_textos):
    documentos = []
    for archivo in os.listdir(carpeta_textos):
        if archivo.endswith(".txt"):
            ruta = os.path.join(carpeta_textos, archivo)
            with open(ruta, "r", encoding="utf-8") as f:
                contenido = f.read()
                documentos.append({"archivo": archivo, "texto": contenido})
    return pd.DataFrame(documentos)

# Ruta a la carpeta de entrenamiento (ajusta según tu estructura)
carpeta_train = "/content/cantemist/background-set"

df_textos = cargar_textos_carpeta(carpeta_train)
print(df_textos.shape)
# Muestra los primeros textos
df_textos#.head()


In [ ]:

# Contar palabras por archivo
df_textos["word_count"] = df_textos["texto"].apply(lambda x: len(x.split()))

# Mostrar cada archivo con su cantidad de palabras
print(df_textos[["archivo", "word_count"]])

# Calcular promedio de palabras por archivo
promedio_palabras = df_textos["word_count"].mean()
print("\nNúmero promedio de palabras por archivo:", promedio_palabras)


In [ ]:
def concatenar_textos(carpeta_textos):
    corpus = []
    for archivo in os.listdir(carpeta_textos):
        if archivo.endswith(".txt"):
            ruta = os.path.join(carpeta_textos, archivo)
            with open(ruta, "r", encoding="utf-8") as f:
                contenido = f.read()
                corpus.append(contenido)
    return " ".join(corpus)

# Ruta a la carpeta de entrenamiento (ajústala a tu entorno)
carpeta_train = "/content/cantemist/background-set"

corpus_general = concatenar_textos(carpeta_train)

print("Longitud total del corpus (caracteres):", len(corpus_general))
print("Primeros 500 caracteres:\n", corpus_general[:500])

In [ ]:

def calcular_textometria(texto):
    return {
        "Flesch Reading Ease": textstat.flesch_reading_ease(texto),
        "Gunning Fog Index": textstat.gunning_fog(texto),
        "Automated Readability Index": textstat.automated_readability_index(texto),
        "SMOG Index": textstat.smog_index(texto),
        "Dale-Chall Readability Score": textstat.dale_chall_readability_score(texto),
        "Fernández-Huerta": textstat.fernandez_huerta(texto),
        "Gutierrez-Polini": textstat.gutierrez_polini(texto),
        "Szigriszt-Pazos": textstat.szigriszt_pazos(texto),
        "Crawford": textstat.crawford(texto),
        "Word Count": textstat.lexicon_count(texto, removepunct=True)  # número de palabras
    }

In [ ]:
calcular_textometria(corpus_general)

In [ ]:
def preprocesar_texto(texto, remover_stopwords=True):
    """
    Preprocesa un texto:
      - Convierte a minúsculas
      - Elimina caracteres no alfanuméricos
      - Tokeniza en palabras
      - (Opcional) elimina stopwords en español
    """
    # 1. Minúsculas
    texto = texto.lower()

    # 2. Eliminar caracteres que no sean letras o números
    texto = re.sub(r'[^a-záéíóúüñ0-9\s]', ' ', texto)

    # 3. Tokenizar (separar por espacios)
    tokens = texto.split()

    # 4. Remover stopwords si se activa
    if remover_stopwords:
        tokens = [t for t in tokens if t not in stopwords_esp]

    # 5. Reconstruir texto limpio
    texto_limpio = " ".join(tokens)

    return texto_limpio

In [ ]:
# FUNCIÓN: Análisis Léxico - Ley de Zipf
def analizar_ley_zipf(texto, titulo):
    tokens = word_tokenize(texto.lower())
    tokens = [t for t in tokens if re.match(r'^\w+$', t)]  # Solo palabras alfanuméricas
    #tokens = [t for t in tokens if t not in stopwords_esp]
    freqs = Counter(tokens)
    df = pd.DataFrame(freqs.items(), columns=['Palabra', 'Frecuencia'])
    df = df.sort_values(by='Frecuencia', ascending=False).reset_index(drop=True)
    df['Rango'] = df.index + 1
    df['Ley_Zipf'] = df['Frecuencia'].iloc[0] / (df['Rango'])
    print("De search: ",df[df["Palabra"]=="de"])
    print()
    print(df.head(20))
    print()

    plt.figure(figsize=(10, 6))
    plt.plot(df['Rango'], df['Frecuencia'], marker='o', label='Datos reales')
    plt.plot(df['Rango'], df['Ley_Zipf'], linestyle='--', label='Zipf teórica')
    plt.title(f"Ley de Zipf - {titulo}")
    plt.xlabel("Rango")
    plt.ylabel("Frecuencia")
    plt.legend()
    plt.grid(True)
    plt.show()

    return df

# EJECUTAR ANÁLISIS LÉXICO
df_zipf_true = analizar_ley_zipf(preprocesar_texto(texto = corpus_general, remover_stopwords=True), "Ley de Zipf (Sin Stopword)")
df_zipf_false = analizar_ley_zipf(preprocesar_texto(texto = corpus_general, remover_stopwords=False), "Ley de Zipf (Con Stopword)")


In [ ]:
print(corpus_general[:500])
print(40*"--")
preprocesar_texto(texto = corpus_general, remover_stopwords=False)[:500]

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
import nltk

# Asegúrate de tener stopwords descargadas
nltk.download("stopwords")
from nltk.corpus import stopwords

def graficos_corpus(corpus_general, top_n=20, remover_stopwords=True):
    # Tokenizar rápido
    tokens = corpus_general.lower().split()

    # Remover stopwords si se activa
    if remover_stopwords:
        stop_es = set(stopwords.words("spanish"))
        tokens = [t for t in tokens if t not in stop_es]

    # --- Gráfico de barras ---
    contador = Counter(tokens)
    palabras_comunes = contador.most_common(top_n)

    palabras, freqs = zip(*palabras_comunes)

    plt.figure(figsize=(10,5))
    plt.bar(palabras, freqs)
    plt.grid()
    plt.xticks(rotation=45)
    plt.title(f"Top {top_n} palabras más frecuentes")
    plt.ylabel("Frecuencia")
    plt.show()

    print("")
    # --- Nube de palabras ---
    wordcloud = WordCloud(width=800, height=400, background_color="white",
                          max_words=200).generate(" ".join(tokens))

    plt.figure(figsize=(10,6))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title("Nube de Palabras del Corpus")
    plt.show()

# Ejemplo de uso:
print("Remover Stopwords")
graficos_corpus(preprocesar_texto(texto = corpus_general, remover_stopwords=True), top_n=20, remover_stopwords=True)
print("Conservar Stopwords")
graficos_corpus(preprocesar_texto(texto = corpus_general, remover_stopwords=False), top_n=20, remover_stopwords=False)



In [ ]:
conteo_doc = df_textos.groupby("archivo")["word_count"].sum()
conteo_doc

In [ ]:
plt.figure(figsize=(15,5))
plt.hist(df_textos["word_count"], bins=20, edgecolor="black")
plt.xticks(range(0, max(conteo_doc)+250, 250))
plt.xlabel("Número total de palabras en documento")
plt.ylabel("Cantidad de documentos")
plt.grid()
plt.title("Histograma de palabras por documento")
plt.show()